In [3]:
import pandas as pd

# 과학적 표기법 비활성화
pd.options.display.float_format = '{:.2f}'.format

In [4]:
df_1 = pd.read_csv('00_CG_orders_250801_260211_shipping_custom.csv')
df_2 = pd.read_csv('Order_earnings_20250801_20260212.csv', skiprows=15)

In [5]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 33 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Order creation date                    81 non-null     object 
 1   Order number                           81 non-null     object 
 2   Item ID                                81 non-null     int64  
 3   Item title                             81 non-null     object 
 4   Buyer name                             81 non-null     object 
 5   Ship to city                           81 non-null     object 
 6   Ship to province/region/state          81 non-null     object 
 7   Ship to zip                            81 non-null     object 
 8   Ship to country                        81 non-null     object 
 9   Transaction currency                   81 non-null     object 
 10  eBay collected tax                     81 non-null     object 
 11  Item pri

In [6]:
df_2 = df_2[['Order creation date', 'Order number', 'Item title', 'Item price', 'Expenses', 'Order earnings']]

df_merge_final = pd.merge(df_1, df_2, how='left', left_on='Order number', right_on='Order number')

df_merge_final = df_merge_final[['Order creation date_x', 'Order number', 'Item title_x', 'Tracking Number', 'Item price', 'Expenses', 'Order earnings', 'Shipping Price', 'Custom tax']]

In [7]:
df_merge_final['Shipping Price'] = df_merge_final['Shipping Price'].astype(str).str.split("\\", n=1).str[-1]

df_merge_final['Shipping Price'] = df_merge_final['Shipping Price'].str.replace(',', '').astype(float)
df_merge_final['Item price'] = df_merge_final['Item price'].astype(str).str.replace(',', '').astype(float)
df_merge_final['Order earnings'] = df_merge_final['Order earnings'].astype(str).str.replace(',', '').astype(float)
df_merge_final['Custom tax'] = df_merge_final['Custom tax'].astype(str).str.replace(',', '').astype(float)

In [8]:
df_merge_final['Purchase Cost'] = (df_merge_final['Item price']*1176-79695)/1.55

df_merge_cleaned = df_merge_final[['Order creation date_x', 'Order number', 'Item title_x', 'Tracking Number', 'Item price','Order earnings', 'Shipping Price', 'Custom tax', 'Purchase Cost']]

df_merge_cleaned['Order earnings'] = df_merge_cleaned['Order earnings']*1470

df_merge_cleaned['Net profit'] = df_merge_cleaned['Order earnings']-df_merge_cleaned['Shipping Price']-df_merge_cleaned['Custom tax']-df_merge_cleaned['Purchase Cost']

C:\Users\ellen\AppData\Local\Temp\ipykernel_29948\634048009.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merge_cleaned['Order earnings'] = df_merge_cleaned['Order earnings']*1470


In [9]:
df_merge_cleaned = df_merge_cleaned.rename(columns={
    'Order earnings' : 'Earning', 'Custom tax' : 'Custom Tax', 'Order creation date_x' : 'Order date',
    'Item title_x' : 'Item Title', 'Item price' : 'Item Price'})

In [15]:
df_merge_cleaned['Order date'] = pd.to_datetime(df_merge_cleaned['Order date'])
df_merge_cleaned = df_merge_cleaned.sort_values(by='Order date', ascending=True)

In [16]:
print(df_merge_cleaned.to_string())

   Order date    Order number                                                                        Item Title  Tracking Number  Item Price     Earning  Shipping Price  Custom Tax  Purchase Cost  Net profit
43 2025-08-14  07-13453-25016                                       Marc_Jacobs wool blend burgundy color skirt     883706302029      114.23   141472.80        27140.00        0.00       35251.28    79081.52
13 2025-08-15  22-13439-84321                          Coach cignature Silver Belt Killer Bee Ornament Handcuff     883735973984      107.31   132799.80        40710.00        0.00       30001.01    62088.79
38 2025-08-16  06-13463-90074                                       Marc_Jacobs C391003 SUSAN purple color 2WAY     883706233389      161.92   199993.50        39550.00        0.00       71434.14    89009.36
6  2025-08-18  21-13449-06816                              Burberry Black Leather Multi Embellished Prosum Tote     883766262737      582.31   720741.00        60110.00

In [17]:
df_merge_cleaned.to_csv('01_CG_check_netprofit.csv', index=False)